In [ ]:
!wget "https://raw.githubusercontent.com/usydnlp/Glocal_K/main/data/MovieLens_1M/movielens_1m_dataset.dat"

--2023-03-20 04:28:34--  https://raw.githubusercontent.com/usydnlp/Glocal_K/main/data/MovieLens_1M/movielens_1m_dataset.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24594131 (23M) [text/plain]
Saving to: ‘movielens_1m_dataset.dat’

movielens_1m_datase 100%[===================>]  23.45M  --.-KB/s    in 0.1s    

2023-03-20 04:28:35 (219 MB/s) - ‘movielens_1m_dataset.dat’ saved [24594131/24594131]



In [ ]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=3241587d03747ae7e109a5213662c1ec4f06d4fe7fadf2532fbd621be5e7db27
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import itertools
from math import sqrt
from operator import add
import time

In [ ]:
spark = SparkSession.builder.appName("ALSExample").getOrCreate()

In [ ]:
spark

In [ ]:
lines = spark.read.text("movielens_1m_dataset.dat").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                      rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training,validation,test) = ratings.randomSplit([0.6, 0.2, 0.2])

In [ ]:
training.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   5.0|978824268|
|     1|     48|   5.0|978824351|
|     1|    150|   5.0|978301777|
|     1|    260|   4.0|978300760|
|     1|    527|   5.0|978824195|
|     1|    531|   4.0|978302149|
|     1|    594|   4.0|978302268|
|     1|    595|   5.0|978824268|
|     1|    661|   3.0|978302109|
|     1|    783|   4.0|978824291|
|     1|    914|   3.0|978301968|
|     1|    919|   4.0|978301368|
|     1|   1022|   5.0|978300055|
|     1|   1028|   5.0|978301777|
|     1|   1029|   5.0|978302205|
|     1|   1097|   4.0|978301953|
|     1|   1197|   3.0|978302268|
|     1|   1207|   4.0|978300719|
|     1|   1270|   5.0|978300055|
|     1|   1287|   5.0|978302039|
+------+-------+------+---------+
only showing top 20 rows



In [ ]:
training.count()

599577

In [ ]:
validation.count()

200888

In [ ]:
test.count()

199744

In [ ]:
ranks = [i for i in range(8, 12)]
lambdas = [((i+1)*0.01) for i in range(5)]
numIters = [i for i in range(8, 12)]
bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [ ]:
total_loop = len(ranks) * len(lambdas) * len(numIters)
count = 1
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
  tic = time.perf_counter()
  # Build the recommendation model using ALS on the training data
  # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
  als = ALS(maxIter=numIter, regParam=lmbda, rank=rank,userCol="userId", itemCol="movieId", ratingCol="rating",
            coldStartStrategy="drop")
  model = als.fit(training)
  predictions = model.transform(validation)
  rmse = evaluator.evaluate(predictions)

  if (rmse < bestValidationRmse):
            bestModel = model
            bestValidationRmse = rmse
            bestRank = rank
            bestLambda = lmbda
            bestNumIter = numIter
  toc = time.perf_counter()
  duration = toc - tic
  print(f"{count} / {total_loop} | RMSE (validation) = {rmse} for the model trained with " \
            f"rank = {rank}, lambda = {lmbda}, and numIter = {numIter} in {int(duration)} seconds")
  count+=1

1 / 80 | RMSE (validation) = 0.9118508889140546 for the model trained with rank = 8, lambda = 0.01, and numIter = 8 in 64 seconds
2 / 80 | RMSE (validation) = 0.9110809089655298 for the model trained with rank = 8, lambda = 0.01, and numIter = 9 in 51 seconds
3 / 80 | RMSE (validation) = 0.9106109478183034 for the model trained with rank = 8, lambda = 0.01, and numIter = 10 in 54 seconds
4 / 80 | RMSE (validation) = 0.9103438165773897 for the model trained with rank = 8, lambda = 0.01, and numIter = 11 in 50 seconds
5 / 80 | RMSE (validation) = 0.8936643987055066 for the model trained with rank = 8, lambda = 0.02, and numIter = 8 in 45 seconds
6 / 80 | RMSE (validation) = 0.892725160780075 for the model trained with rank = 8, lambda = 0.02, and numIter = 9 in 47 seconds
7 / 80 | RMSE (validation) = 0.8920375916962698 for the model trained with rank = 8, lambda = 0.02, and numIter = 10 in 47 seconds
8 / 80 | RMSE (validation) = 0.8914996003304769 for the model trained with rank = 8, lam

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = bestModel.transform(test)
rmse = evaluator.evaluate(predictions)
print(f"RMSE (validation) = {rmse} for the model trained with " \
            f"rank ={bestRank}, lambda = {bestLambda}, and numIter = {bestNumIter}.")

# Generate top 10 movie recommendations for each user
userRecs = bestModel.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = bestModel.recommendForAllItems(10)

# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = bestModel.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = bestModel.recommendForItemSubset(movies, 10)
# $example off$
userRecs.show()
movieRecs.show()
userSubsetRecs.show()
movieSubSetRecs.show()

RMSE (validation) = 0.8720499411343874 for the model trained with rank =10, lambda = 0.05, and numIter = 11.
+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{1421, 6.389059}...|
|     3|[{3847, 5.9229603...|
|     5|[{850, 5.397816},...|
|     6|[{831, 5.9605584}...|
|     9|[{2309, 4.8958163...|
|    12|[{1421, 5.3103156...|
|    13|[{572, 4.808655},...|
|    15|[{572, 4.9886327}...|
|    16|[{1930, 6.488145}...|
|    17|[{572, 5.7920113}...|
|    19|[{572, 5.0061007}...|
|    20|[{1117, 6.353433}...|
|    22|[{557, 4.5964837}...|
|    26|[{572, 5.0878906}...|
|    27|[{1450, 6.004456}...|
|    28|[{557, 4.7801194}...|
|    31|[{572, 5.278276},...|
|    34|[{572, 5.733081},...|
|    35|[{1450, 4.923674}...|
|    37|[{1169, 6.085557}...|
+------+--------------------+
only showing top 20 rows

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|      1|[{3373, 5.978347}...|
|      3|[{3915, 5.18